<center>

**МИНИСТЕРСТВО ОБРАЗОВАНИЯ И НАУКИ РОССИЙСКОЙ ФЕДЕРАЦИИ**

Федеральное государственное автономное образовательное учреждение  
высшего профессионального образования  
**«Дальневосточный федеральный университет»**  
*(ДВФУ)*  

---
<br><br>

**Лабораторная работа № 2**  
по дисциплине *«Методы оптимизации»*  
Направление подготовки **01.03.02 Прикладная математика и информатика**  
Очной формы обучения  

на тему  

### "Реализация метода градиентного спуска"

</center>


<br><br>


<div style="text-align: left; margin-left: 50%;">

Студентка группы **Б9123-01.03.02сп**  
Бобышева Ю.В. ______________________ (подпись)  

Преподаватель  
______________________ /Неверова Г.П./  

</div>

<br><br><br><br>

<center>


Владивосток  
**2025 г.**

</center>


In [1]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
pd.set_option("display.precision", 14)

## Метод градиентного спуска

**Градиентный спуск (Gradient Descent)** — это итерационный метод оптимизации, используемый для поиска минимума дифференцируемой функции $f(x)$.  
Основная идея: двигаться в направлении, **противоположном градиенту**, поскольку градиент указывает направление **наибольшего роста** функции.

$$x^{(k+1)} = x^{(k)} - h \, \nabla f(x^{(k)})$$

где  
- $x^{(k)}$ — вектор параметров на шаге $k$;  
- $h > 0$ — **шаг (learning rate)**;  
- $\nabla f(x^{(k)})$ — градиент функции в текущей точке.

---

### Алгоритм

1. **Задать начальное приближение** $x^{(0)}$;  
2. **Повторять до сходимости:**
   - Вычислить градиент: $g^{(k)} = \nabla f(x^{(k)})$;  
   - Обновить точку: $x^{(k+1)} = x^{(k)} - h \, g^{(k)}$;  
3. **Проверить критерий остановки.**

---

### Критерии остановки

Метод можно завершать, если выполнено хотя бы одно из условий:

1. **Норма градиента мала:**
   $$\|\nabla f(x^{(k)})\| < \varepsilon_1$$
   
или

2. **Изменение аргумента мало и изменение значения функции мало:**
   $$\|x^{(k+1)} - x^{(k)}\| < \varepsilon_2$$
   
    $$|f(x^{(k+1)}) - f(x^{(k)})| < \varepsilon_3$$
---

### Выбор шага $h$

- Если $h$ слишком **мал**, метод сходится **медленно**.  
- Если $h$ слишком **большой**, возможны **осцилляции** или **расходимость**.  
- Оптимальный шаг можно выбирать:
  - фиксированно (например, $h = 0.01$);
  - адаптивно, например: 

$$\text{если } f(x^{(k+1)}) > f(x^{(k)}), $$

$$\text{ то } h = \frac{h}{2}$$


In [2]:
def f(A, b, x):
    return 0.5 * (x.T @ A @ x) + b @ x

def grad(A, b, x):
    return A @ x + b

def gradient_descent(A : np.ndarray, b: np.ndarray, start_point : np.ndarray = None, h : float = 0.1, h_denom : float = 2, max_iter : int = 10000, eps : float = 0.00001):
    
    if start_point is None:
        x = np.zeros_like(b)
    else:
        x = np.copy(start_point)
        
    gradient = grad(A, b, x)
    xs = [x.copy()]
    
    for i in range(max_iter):
        x_next = x - h * gradient
        gradient = grad(A, b, x_next)
        xs.append(x_next.copy())
        
        if np.linalg.norm(gradient) < eps:
            x = x_next
            break
            
        if f(A, b, x_next) > f(A, b, x):
            h /= h_denom
            
        x = x_next
    
    return x, np.array(xs)

Генерируем положительно определенную матрицу:

In [3]:
np.random.seed(42)
M = np.random.randn(3, 3)
A = M.T @ M + np.eye(3) * 0.5
b = np.random.randn(3)

eigvals = np.linalg.eigvalsh(A)
print(A)
print("Собственные значения:", eigvals)

[[ 5.56025801  0.78664234 -0.77628148]
 [ 0.78664234  1.16290088 -0.39501919]
 [-0.77628148 -0.39501919  1.19472676]]
Собственные значения: [0.78320384 1.28854214 5.84613967]


In [4]:
x_gd, xes_ = gradient_descent(A, b, [-4, -4, -4])

values = np.array([f(A, b, x) for x in xes_])
grad_values = np.array([np.linalg.norm(grad(A, b, x)) for x in xes_])
df = pd.DataFrame(
    np.hstack([xes_, values[:, None], grad_values[:, None]]),
    columns=['x1', 'x2', 'x3', 'f(x)', 'gradient(x)']
)

df

,x1,x2,x3,f(x),gradient(x)
0,-4.00000000000000,-4.00000000000000,-4.00000000000000,58.73490155191288,22.75037400737559
1,-1.82600845465346,-3.33184861707798,-3.94405659095260,21.41141349550884,10.38448829316347
2,-0.90902913339936,-2.91020212585069,-3.69964069143584,13.19384300245208,5.85159443119215
3,-0.51610891853894,-2.60006752141818,-3.39658656135490,10.27602525875253,4.37118618935221
4,-0.34253345123939,-2.34493604561038,-3.08698653838378,8.50608322905324,3.80093830026476
...,...,...,...,...,...
158,-0.11641887472582,0.65784731217367,0.53167763002624,-0.30782561612882,0.00001281900220
159,-0.11641888643396,0.65784824787099,0.53167850615501,-0.30782561614461,0.00001181501303
160,-0.11641889722573,0.65784911028576,0.53167931366329,-0.30782561615802,0.00001088965667
161,-0.11641890717283,0.65784990515730,0.53168005792569,-0.30782561616942,0.00001003677458


### Понадобилось 162 итерации, чтобы сработал критерий остановки градиентного спуска

### Аналитическое решение:

Нам дана функция $f(x) = \frac{1}{2} (x^T A x) + Bx$

Её градиент равен $\bigtriangledown f(x) = Ax + B$

Минимального значения функция достигает в точке, где градиент обращается в ноль, то есть $Ax + B = 0$, 

так как положительно определенная матрица является невырожденной:

$$x = -A^{-1}B$$



In [5]:
x = -np.linalg.solve(A, b)
x

array([-0.11641902,  0.65785926,  0.53168882])

### Решение совпало с конечным приближением до 5 знака после запятой

In [7]:
x1 = np.linspace(-4, 4, 200)
x2 = np.linspace(-4, 4, 200)
X1, X2 = np.meshgrid(x1, x2)

Z = np.zeros_like(X1)
for i in range(X1.shape[0]):
    for j in range(X1.shape[1]):
        x_vec = np.array([X1[i, j], X2[i, j], 0])
        Z[i, j] = f(A, b, x_vec)

# Траектория
xes2d = xes_[:, :2]
Z_path = np.array([f(A, b, np.append(x, 0)) for x in xes2d])

fig = go.Figure()

# Поверхность функции
fig.add_trace(go.Surface(
    x=X1, y=X2, z=Z,
    colorscale='Viridis',
    opacity=0.85,
    showscale=True,
    colorbar=dict(title="f(x)")
))

fig.add_trace(go.Contour(
    x=x1, y=x2, z=Z,
    contours=dict(showlines=False),
    showscale=False,
    colorscale='Viridis',
    opacity=0.5
))

# Траектория
fig.add_trace(go.Scatter3d(
    x=xes2d[:,0],
    y=xes2d[:,1],
    z=Z_path,
    mode='lines+markers',
    line=dict(
        width=6,
        color='yellow',
        colorscale='Turbo'
    ),
    marker=dict(size=4, color='red'),
    name='Путь градиентного спуска'
))

fig.update_layout(
    scene=dict(
        xaxis_title='x₁',
        yaxis_title='x₂',
        zaxis_title='f(x)',
        aspectmode='auto'
    ),
    title='Поверхность с траекторией градиентного спуска',
    template='plotly_white'
)

fig.show()